<a href="https://colab.research.google.com/github/sarasabino/Sistema_Recomendacion_BBDD_Explotaciones/blob/main/MVP_data_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP - BBDD Generation recommender system

One of the main difficulties on the implementation of this project is the fact that the data needs to be standarized in order to be able to compare. 

For the purpose to present the idea in a clear way this notebook present a simple example of how the recommender system will proceed once the data is all standarized. 

This problem with the data is not going to be present once this previous work is done because once the project is in "pro" version the user will feed the system with their answers in a standarized way. It will be only be relevant to keep going the recommender as first place. 

The data use in this notebook is totally manipulated to be able to compare at first without doing the standarizing exercise as it will be taking long.

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os
import time

In [5]:
from google.colab import files
uploaded = files.upload()

Saving projects_with_tables.csv to projects_with_tables.csv


In [104]:
test_file = pd.read_csv(io.BytesIO(uploaded['projects_with_tables.csv']), sep = ';', header=None)

In [105]:
test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,RRHH,Plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Brecha Salarial,Plantilla_mensual,Brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RRHH,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,RRHH,Entidades RRHH,Indicadores RRHH,Periodos Informes RRHH,Valores Informe RRHH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,RRHH,Indicadores RSC,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,Responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_MES,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades


In [106]:
#We set all text in the df to lowercase
test_file = test_file.apply(lambda x : x.str.lower())

test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,rrhh,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,brecha salarial,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rrhh,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,rrhh,entidades rrhh,indicadores rrhh,periodos informes rrhh,valores informe rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rrhh,indicadores rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades


In [107]:
#We create a new column with a list containing all tables in that project

test_file['tables'] = test_file.loc[:,test_file.columns != 0].values.tolist()

test_file.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,tables
0,rrhh,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[plantilla_mensual, tipo_contrato, categoria_p..."
1,brecha salarial,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[plantilla_mensual, brecha_salarios, tipo_cont..."
2,rrhh,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN,"[rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh..."
3,rrhh,entidades rrhh,indicadores rrhh,periodos informes rrhh,valores informe rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[entidades rrhh, indicadores rrhh, periodos in..."
4,rrhh,indicadores rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,"[indicadores rsc, rrhh_altas, rrhh_bajas, r_rh..."


We drop the individual table name columns in order to get a df with only the project name and the tables list

In [108]:
test_file.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 'tables'], dtype='object')

In [109]:
project_with_tables = test_file.drop(columns = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14])

In [110]:
project_with_tables_nan = project_with_tables.rename(columns={0:'Project'})

In [111]:
project_with_tables_nan.head()

,Project,tables
0,rrhh,"[plantilla_mensual, tipo_contrato, categoria_p..."
1,brecha salarial,"[plantilla_mensual, brecha_salarios, tipo_cont..."
2,rrhh,"[rrhh_absentismo, rrhh_altas, rrhh_bajas, rhhh..."
3,rrhh,"[entidades rrhh, indicadores rrhh, periodos in..."
4,rrhh,"[indicadores rsc, rrhh_altas, rrhh_bajas, r_rh..."


In [112]:
project_with_tables_nan['tables'].isnull().values.any()

False

Creating a dictionary for projects and table names where index will be the type of project and the values the list of tables

In [113]:
tables_dict_nan = {}

for index, row in project_with_tables_nan.iterrows():
    tables_dict_nan[index] = row[1]

In [114]:
tables_dict_nan

{0: ['plantilla_mensual',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 1: ['plantilla_mensual',
  'brecha_salarios',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 2: ['rrhh_absentismo',
  'rrhh_altas',
  'rrhh_bajas',
  'rhhh_plantilla',
  'rrhh_contratacion',
  'rrhh_indicadores',
  'rrhh_jornada_trabajo',
  'rrhhh_datos',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 3: ['entidades rrhh',
  'indicadores rrhh',
  'periodos informes rrhh',
  'valores informe rrhh',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan],
 4: ['indicadores rsc',
  'rrhh_altas',
  'rrhh_bajas',
  'r_rhh_tipo_entidad_juridica',
  'responsables de sociedades juridicas',
  'rrhh_carga_indicadores',
  'rrhh_carga_indicadores_mes',
  nan,
  'u_rhh_carga_subactividades',

In [115]:
project_id_dict = {}

for index, row in project_with_tables_nan.iterrows():
    project_id_dict[index] = row[0]

In [116]:
project_id_dict

{0: 'rrhh',
 1: 'brecha salarial',
 2: 'rrhh',
 3: 'rrhh',
 4: 'rrhh',
 5: 'brecha salarial',
 6: 'rrhh',
 7: 'seguridad y salud',
 8: 'canal negocio responsable',
 9: 'operaciones'}

We remove all the NaN elements in the list inside the dictionary

In [117]:
tablesdict = {}
count=0
for list_ in tables_dict_nan.values():
    vector=[]
    
    for item in list_:
        
        if str(item) != 'nan':
            vector.append(item)
            
    tablesdict[count]= vector
    count+=1  

In [118]:
tablesdict

{0: ['plantilla_mensual',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo'],
 1: ['plantilla_mensual',
  'brecha_salarios',
  'tipo_contrato',
  'categoria_profesional',
  'empleados_excluidos',
  'nacionalidad',
  'pais_trabajo'],
 2: ['rrhh_absentismo',
  'rrhh_altas',
  'rrhh_bajas',
  'rhhh_plantilla',
  'rrhh_contratacion',
  'rrhh_indicadores',
  'rrhh_jornada_trabajo',
  'rrhhh_datos'],
 3: ['entidades rrhh',
  'indicadores rrhh',
  'periodos informes rrhh',
  'valores informe rrhh'],
 4: ['indicadores rsc',
  'rrhh_altas',
  'rrhh_bajas',
  'r_rhh_tipo_entidad_juridica',
  'responsables de sociedades juridicas',
  'rrhh_carga_indicadores',
  'rrhh_carga_indicadores_mes',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades',
  'u_rhh_carga_subactividades'],
 5: ['brecha salarial',
  'brecha ajustada',
  'ids a excluir 

We create a df with all types of tables

In [119]:
all_tables = pd.DataFrame()
vector = []

for list_ in tablesdict.values():
   for item in list_: 
    vector.append(item)


In [120]:
all_tables = all_tables.append(vector)

all_tables

,0
0,plantilla_mensual
1,tipo_contrato
2,categoria_profesional
3,empleados_excluidos
4,nacionalidad
...,...
72,preguntas
73,respuestas
74,resultados
75,planes de accion


In [121]:
all_tables = all_tables.drop_duplicates()

In [122]:
id_tables = {}

count=0

for index, row in all_tables.iterrows():
  id_tables[count] = row[0]
  count+=1

In [123]:
id_tables

{0: 'plantilla_mensual',
 1: 'tipo_contrato',
 2: 'categoria_profesional',
 3: 'empleados_excluidos',
 4: 'nacionalidad',
 5: 'pais_trabajo',
 6: 'brecha_salarios',
 7: 'rrhh_absentismo',
 8: 'rrhh_altas',
 9: 'rrhh_bajas',
 10: 'rhhh_plantilla',
 11: 'rrhh_contratacion',
 12: 'rrhh_indicadores',
 13: 'rrhh_jornada_trabajo',
 14: 'rrhhh_datos',
 15: 'entidades rrhh',
 16: 'indicadores rrhh',
 17: 'periodos informes rrhh',
 18: 'valores informe rrhh',
 19: 'indicadores rsc',
 20: 'r_rhh_tipo_entidad_juridica',
 21: 'responsables de sociedades juridicas',
 22: 'rrhh_carga_indicadores',
 23: 'rrhh_carga_indicadores_mes',
 24: 'u_rhh_carga_subactividades',
 25: 'brecha salarial',
 26: 'brecha ajustada',
 27: 'ids a excluir brecha',
 28: 'percentiles brecha salarial',
 29: 'matriculas a obviar brecha',
 30: 'permisos brecha salarial',
 31: 'import brecha',
 32: 'plantilla_anual',
 33: 'movimientos plantilla unica',
 34: 'movimientos cierre mes',
 35: 'tipo de modificacion',
 36: 'versiones 

In [124]:
# Define a function that retrieves all the ids and names for tables containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in tables name
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in tables name, and an empty list otherwise.
    """
    # convert input text to lowercase

    text_ = text.lower()
    
    # find occurances
    search = [(k, v .lower().find(text_)) for k, v in list(ids.items())]
    
    # Get the IDs corresponding to the given text
    index = [k for k, v in search if v > -1]
    
    # Return a list with the id and the name
    out = []
    for i in index:
        out.append((i, ids[i]))
        
    
    return out

In [125]:
returnItemId('altas', id_tables)

[(8, 'rrhh_altas')]

In [126]:
print(len(set(id_tables.keys())))
print(len(set(id_tables.values())))

62
62


We create a df with the dictionary with the columns as the index of the projects

In [145]:
projects = pd.DataFrame.from_dict(tables_dict_nan, orient='index').reset_index()

projects

,index,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0,plantilla_mensual,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,plantilla_mensual,brecha_salarios,tipo_contrato,categoria_profesional,empleados_excluidos,nacionalidad,pais_trabajo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rrhh_absentismo,rrhh_altas,rrhh_bajas,rhhh_plantilla,rrhh_contratacion,rrhh_indicadores,rrhh_jornada_trabajo,rrhhh_datos,NaN,NaN,NaN,NaN,NaN,NaN
3,3,entidades rrhh,indicadores rrhh,periodos informes rrhh,valores informe rrhh,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,indicadores rsc,rrhh_altas,rrhh_bajas,r_rhh_tipo_entidad_juridica,responsables de sociedades juridicas,rrhh_carga_indicadores,rrhh_carga_indicadores_mes,NaN,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades,u_rhh_carga_subactividades
5,5,brecha salarial,brecha ajustada,ids a excluir brecha,percentiles brecha salarial,matriculas a obviar brecha,permisos brecha salarial,import brecha,rrhh_altas,NaN,NaN,NaN,NaN,NaN,NaN
6,6,plantilla_anual,movimientos plantilla unica,movimientos cierre mes,tipo de modificacion,versiones plantilla unica,sbq movimientos periodo anterios,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,d_hs_causa,tipo_contrato,d_hs_empresa,d_hs_presentado,d_hs_tareas,d_hs_territorio,d_hs_tipo_accidente_externo,d_hs_unidad,t_hs_accidentes_externos,t_hs_comunicacion_investigacion_accidentes,NaN,NaN,NaN,NaN
8,8,canal gestion responsable,criticidad,delegaciones,emails del gestor,logs de emails,plantillas emails,tipologias de consultas comerciales,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,gen_cuestionario,gen_ambito1,gestdoc_ambito2,preguntas,respuestas,resultados,planes de accion,palancas,NaN,NaN,NaN,NaN,NaN,NaN


In [169]:
dummies_projects = pd.get_dummies(projects)

dummies_projects

,index,0_brecha salarial,0_canal gestion responsable,0_d_hs_causa,0_entidades rrhh,0_gen_cuestionario,0_indicadores rsc,0_plantilla_anual,0_plantilla_mensual,0_rrhh_absentismo,1_brecha ajustada,1_brecha_salarios,1_criticidad,1_gen_ambito1,1_indicadores rrhh,1_movimientos plantilla unica,1_rrhh_altas,1_tipo_contrato,2_categoria_profesional,2_d_hs_empresa,2_delegaciones,2_gestdoc_ambito2,2_ids a excluir brecha,2_movimientos cierre mes,2_periodos informes rrhh,2_rrhh_bajas,2_tipo_contrato,3_categoria_profesional,3_d_hs_presentado,3_emails del gestor,3_empleados_excluidos,3_percentiles brecha salarial,3_preguntas,3_r_rhh_tipo_entidad_juridica,3_rhhh_plantilla,3_tipo de modificacion,3_valores informe rrhh,4_d_hs_tareas,4_empleados_excluidos,4_logs de emails,4_matriculas a obviar brecha,4_nacionalidad,4_responsables de sociedades juridicas,4_respuestas,4_rrhh_contratacion,4_versiones plantilla unica,5_d_hs_territorio,5_nacionalidad,5_pais_trabajo,5_permisos brecha salarial,5_plantillas emails,5_resultados,5_rrhh_carga_indicadores,5_rrhh_indicadores,5_sbq movimientos periodo anterios,6_d_hs_tipo_accidente_externo,6_import brecha,6_pais_trabajo,6_planes de accion,6_rrhh_carga_indicadores_mes,6_rrhh_jornada_trabajo,6_tipologias de consultas comerciales,7_d_hs_unidad,7_palancas,7_rrhh_altas,7_rrhhh_datos,8_t_hs_accidentes_externos,8_u_rhh_carga_subactividades,9_t_hs_comunicacion_investigacion_accidentes,9_u_rhh_carga_subactividades,10_u_rhh_carga_subactividades,11_u_rhh_carga_subactividades,12_u_rhh_carga_subactividades,13_u_rhh_carga_subactividades
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,1,1,1
5,5,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0
8,8,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
9,9,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [166]:
projects_reset = pd.DataFrame.from_dict(tables_dict_nan)

projects_reset

,0,1,2,3,4,5,6,7,8,9
0,plantilla_mensual,plantilla_mensual,rrhh_absentismo,entidades rrhh,indicadores rsc,brecha salarial,plantilla_anual,d_hs_causa,canal gestion responsable,gen_cuestionario
1,tipo_contrato,brecha_salarios,rrhh_altas,indicadores rrhh,rrhh_altas,brecha ajustada,movimientos plantilla unica,tipo_contrato,criticidad,gen_ambito1
2,categoria_profesional,tipo_contrato,rrhh_bajas,periodos informes rrhh,rrhh_bajas,ids a excluir brecha,movimientos cierre mes,d_hs_empresa,delegaciones,gestdoc_ambito2
3,empleados_excluidos,categoria_profesional,rhhh_plantilla,valores informe rrhh,r_rhh_tipo_entidad_juridica,percentiles brecha salarial,tipo de modificacion,d_hs_presentado,emails del gestor,preguntas
4,nacionalidad,empleados_excluidos,rrhh_contratacion,NaN,responsables de sociedades juridicas,matriculas a obviar brecha,versiones plantilla unica,d_hs_tareas,logs de emails,respuestas
5,pais_trabajo,nacionalidad,rrhh_indicadores,NaN,rrhh_carga_indicadores,permisos brecha salarial,sbq movimientos periodo anterios,d_hs_territorio,plantillas emails,resultados
6,NaN,pais_trabajo,rrhh_jornada_trabajo,NaN,rrhh_carga_indicadores_mes,import brecha,NaN,d_hs_tipo_accidente_externo,tipologias de consultas comerciales,planes de accion
7,NaN,NaN,rrhhh_datos,NaN,NaN,rrhh_altas,NaN,d_hs_unidad,NaN,palancas
8,NaN,NaN,NaN,NaN,u_rhh_carga_subactividades,NaN,NaN,t_hs_accidentes_externos,NaN,NaN
9,NaN,NaN,NaN,NaN,u_rhh_carga_subactividades,NaN,NaN,t_hs_comunicacion_investigacion_accidentes,NaN,NaN


In [146]:
projects_array = projects.iloc[:, :].values

In [147]:
projects_array

array([[0, 'plantilla_mensual', 'tipo_contrato', 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan, nan],
       [1, 'plantilla_mensual', 'brecha_salarios', 'tipo_contrato',
        'categoria_profesional', 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan],
       [2, 'rrhh_absentismo', 'rrhh_altas', 'rrhh_bajas',
        'rhhh_plantilla', 'rrhh_contratacion', 'rrhh_indicadores',
        'rrhh_jornada_trabajo', 'rrhhh_datos', nan, nan, nan, nan, nan,
        nan],
       [3, 'entidades rrhh', 'indicadores rrhh',
        'periodos informes rrhh', 'valores informe rrhh', nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan],
       [4, 'indicadores rsc', 'rrhh_altas', 'rrhh_bajas',
        'r_rhh_tipo_entidad_juridica',
        'responsables de sociedades juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactivid

In [137]:
from sklearn.preprocessing import LabelEncoder

In [148]:
table = LabelEncoder()

In [154]:
projects_array

array([[0, 7, 'tipo_contrato', 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan, nan],
       [1, 7, 'brecha_salarios', 'tipo_contrato',
        'categoria_profesional', 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan],
       [2, 8, 'rrhh_altas', 'rrhh_bajas', 'rhhh_plantilla',
        'rrhh_contratacion', 'rrhh_indicadores', 'rrhh_jornada_trabajo',
        'rrhhh_datos', nan, nan, nan, nan, nan, nan],
       [3, 3, 'indicadores rrhh', 'periodos informes rrhh',
        'valores informe rrhh', nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan],
       [4, 5, 'rrhh_altas', 'rrhh_bajas', 'r_rhh_tipo_entidad_juridica',
        'responsables de sociedades juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades',
        'u_rhh_carga_sub

In [150]:
projects_array[:, 1]= table.fit_transform(projects_array[:, 1]) #vectorizamos la primera columna

In [158]:
projects_array[:,3]= table.fit_transform(projects_array[:,3])

In [159]:
projects_array[:,2:]

array([['tipo_contrato', 0, 'empleados_excluidos', 'nacionalidad',
        'pais_trabajo', nan, nan, nan, nan, nan, nan, nan, nan],
       ['brecha_salarios', 8, 'categoria_profesional',
        'empleados_excluidos', 'nacionalidad', 'pais_trabajo', nan, nan,
        nan, nan, nan, nan, nan],
       ['rrhh_altas', 7, 'rhhh_plantilla', 'rrhh_contratacion',
        'rrhh_indicadores', 'rrhh_jornada_trabajo', 'rrhhh_datos', nan,
        nan, nan, nan, nan, nan],
       ['indicadores rrhh', 6, 'valores informe rrhh', nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan],
       ['rrhh_altas', 7, 'r_rhh_tipo_entidad_juridica',
        'responsables de sociedades juridicas', 'rrhh_carga_indicadores',
        'rrhh_carga_indicadores_mes', nan, 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades', 'u_rhh_carga_subactividades',
        'u_rhh_carga_subactividades'],
       ['brecha ajustada', 4, 'percentiles b